In [1]:
# import the usual libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from scikeras.wrappers import KerasRegressor
from keras.utils import to_categorical
# from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

# other machine learning modules
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# commonly used modules
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

print(tf.__version__)


2023-11-17 11:39:44.690889: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 11:39:44.850121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 11:39:44.850182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 11:39:44.860836: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 11:39:44.890141: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 11:39:44.890541: I tensorflow/core/platform/cpu_feature_guard.cc:1

2.15.0


In [ ]:
excludeYears = [2000, 2001]

df_attendance = pd.read_csv('attendance.csv')
df_attendance = df_attendance[~df_attendance['year'].isin(excludeYears)]

df_games = pd.read_csv('games.csv')
df_games = df_games[~df_games['year'].isin(excludeYears)]

df_standings = pd.read_csv('standings.csv')

In [ ]:
# TODO: Data team will handle putting in the training, validation, and test data
X = df_attendance.drop('weekly_attendance', axis=1)
Y = df_attendance['weekly_attendance']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# X_valid = 0??

# Y_valid = 0??


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # TODO: need training data
X_test = scaler.transform(X_test)

In [ ]:
def create_model(optimizer='adam', activation='relu', neurons=64):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [ ]:
model = KerasRegressor(build_fn=create_model, verbose=0)

In [ ]:
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'neurons': [32, 64, 128]
}

In [ ]:
scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scorer, cv=5)
grid_result = grid.fit(X_train, Y_train)

In [ ]:
best_params = grid_result.best_params_
print("Best Hyperparameters:", best_params)

In [ ]:
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
print("Final Mean Squared Error on Test Set:", mse)